In [1]:
#Doc https://documentation.dataspace.copernicus.eu/notebook-samples/sentinelhub/introduction_to_SH_APIs.html
#API KEY : https://shapps.dataspace.copernicus.eu/dashboard/#/

In [2]:
#pip install sentinelhub

In [3]:
# Utilities
import matplotlib.pyplot as plt
import pandas as pd
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

#from utils import plot_image

c:\Users\m\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

from dotenv import load_dotenv
import os

# Carga las variables del archivo .env (en el directorio raíz del proyecto)
load_dotenv(dotenv_path='../.env')  # Ajusta la ruta si tu notebook está en una subcarpeta

client_id = os.environ.get('CDSE_CLIENT_ID')
client_secret = os.environ.get('CDSE_CLIENT_SECRET')

In [5]:
# Only run this cell if you have not created a configuration.

config = SHConfig()
config.sh_client_id = client_id
config.sh_client_secret = client_secret
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

In [6]:

# Definir el área de interés (AOI) con 4 puntos de lat/lon
lon_min = -63.95
lat_min = -31.75
lon_max = -63.75
lat_max = -31.60

aoi_coords_wgs84 = [lon_min, lat_min, lon_max, lat_max]

In [7]:
resolution = 10
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {aoi_size} pixels")

Image shape at 10 m resolution: (1883, 1677) pixels


In [8]:
catalog = SentinelHubCatalog(config=config)
catalog

In [9]:
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
time_interval = "2025-07-01", "2025-07-20"

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L2A,
    bbox=aoi_bbox,
    time=time_interval,
    fields={"include": ["id", "properties.datetime"], "exclude": []},
)

results = list(search_iterator)
print("Total number of results:", len(results))

results

Total number of results: 10


[{'id': 'S2A_MSIL2A_20250719T141111_N0511_R110_T20HMK_20250719T211521.SAFE',
  'properties': {'datetime': '2025-07-19T14:21:58.521Z'}},
 {'id': 'S2A_MSIL2A_20250719T141111_N0511_R110_T20JML_20250719T211521.SAFE',
  'properties': {'datetime': '2025-07-19T14:21:44.092Z'}},
 {'id': 'S2C_MSIL2A_20250717T141111_N0511_R110_T20HMK_20250717T173322.SAFE',
  'properties': {'datetime': '2025-07-17T14:22:02.422Z'}},
 {'id': 'S2C_MSIL2A_20250717T141111_N0511_R110_T20JML_20250717T173322.SAFE',
  'properties': {'datetime': '2025-07-17T14:21:47.995Z'}},
 {'id': 'S2B_MSIL2A_20250712T140709_N0511_R110_T20HMK_20250712T173831.SAFE',
  'properties': {'datetime': '2025-07-12T14:21:42.721Z'}},
 {'id': 'S2B_MSIL2A_20250712T140709_N0511_R110_T20JML_20250712T173831.SAFE',
  'properties': {'datetime': '2025-07-12T14:21:28.273Z'}},
 {'id': 'S2C_MSIL2A_20250707T141111_N0511_R110_T20HMK_20250707T191716.SAFE',
  'properties': {'datetime': '2025-07-07T14:22:01.949Z'}},
 {'id': 'S2C_MSIL2A_20250707T141111_N0511_R110_T

In [10]:
#!/usr/bin/env python3
"""
Descarga robusta de tiles completos Sentinel-2 (L2A) desde Copernicus Data Space (ODATA).
Incluye filtro por plataforma (ej: 'S2A') + post-filtrado por fecha de sensado extraída del NAME.
"""

import os
import re
import zipfile
import glob
import shutil
from pathlib import Path
from datetime import datetime, date
import requests
import rasterio
import numpy as np

# ----------------------
# CONFIGURACIÓN (modifica aquí)
# ----------------------
CLIENT_ID = os.environ.get("CDSE_CLIENT_ID", client_id)
CLIENT_SECRET = os.environ.get("CDSE_CLIENT_SECRET", client_secret)

AOI = [-63.95, -31.75, -63.75, -31.6]
START_DATE = "2024-07-01"
END_DATE   = "2024-07-20"
MAX_CLOUD_COVER = 20
MAX_PRODUCTS_TO_PROCESS = 10
DOWNLOAD_DIR = Path("sentinel2_complete_tiles")
PROCESSED_DIR = Path("processed_tiles")
TOP_N = 200

# aquí elegís la plataforma: "S2A", "S2B", "S2C" o None para cualquiera
PLATFORM = "S2A"
# ----------------------

def get_cdse_access_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET):
    token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    }
    resp = requests.post(token_url, data=data, timeout=30)
    if resp.status_code == 200:
        print("✓ Access token obtained successfully")
        return resp.json().get('access_token')
    else:
        print(f"✗ Failed to get access token: {resp.status_code}")
        print(resp.text)
        return None

def _parse_sensing_datetime_from_name(name):
    m = re.search(r'_(\d{8}T\d{6})_', name)
    if not m:
        return None
    try:
        return datetime.strptime(m.group(1), "%Y%m%dT%H%M%S")
    except Exception:
        return None

def build_odata_query(aoi, start_date, end_date, platform=None, attempt_sensing_filter=True, top=TOP_N):
    lon_min, lat_min, lon_max, lat_max = aoi
    wkt_polygon = f"POLYGON(({lon_min} {lat_min},{lon_max} {lat_min},{lon_max} {lat_max},{lon_min} {lat_max},{lon_min} {lat_min}))"
    base_url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products"

    sensing_field_candidates = ["SensingTime", "SensingStart", "SensingStartDate", "SensingStartDateTime"]
    content_filter = f"ContentDate/Start ge {start_date}T00:00:00.000Z and ContentDate/Start le {end_date}T23:59:59.999Z"

    filters = [
        "Collection/Name eq 'SENTINEL-2'",
        "contains(Name,'MSIL2A')",
        f"OData.CSC.Intersects(area=geography'SRID=4326;{wkt_polygon}')",
    ]

    # filtro por plataforma (ej: startswith(Name,'S2A_'))
    if platform:
        # usamos startswith para reducir resultados; si el servicio no soporta startswith, el servidor podría retornar error o ignorarlo
        filters.append(f"startswith(Name,'{platform}_') eq true")

    if attempt_sensing_filter:
        sensing_candidate = sensing_field_candidates[0]
        sensing_filter = f"{sensing_candidate} ge {start_date}T00:00:00.000Z and {sensing_candidate} le {end_date}T23:59:59.999Z"
        filters.append(sensing_filter)
    else:
        filters.append(content_filter)

    filter_string = " and ".join(filters)
    url = f"{base_url}?$filter={filter_string}&$orderby=ContentDate/Start desc&$top={top}"
    return url

def search_sentinel2_products_odata(aoi, start_date, end_date, access_token=None, platform=None, top=TOP_N):
    print("🔍 Searching for Sentinel-2 products via OData...")
    url = build_odata_query(aoi, start_date, end_date, platform=platform, attempt_sensing_filter=True, top=top)
    headers = {}
    if access_token:
        headers['Authorization'] = f'Bearer {access_token}'

    try:
        resp = requests.get(url, headers=headers, timeout=60)
    except Exception as e:
        print("✗ Error contacting OData endpoint:", e)
        return []

    if resp.status_code != 200:
        print("⚠️ OData returned non-200. Intentando fallback usando ContentDate...")
        url = build_odata_query(aoi, start_date, end_date, platform=platform, attempt_sensing_filter=False, top=top)
        try:
            resp = requests.get(url, headers=headers, timeout=60)
        except Exception as e:
            print("✗ Fallback request failed:", e)
            return []
        if resp.status_code != 200:
            print("✗ Fallback OData also failed:", resp.status_code)
            print(resp.text)
            return []

    data = resp.json()
    products = data.get('value', [])
    print(f"✓ OData returned {len(products)} products (before post-filtering)")

    # POST-FILTER por fecha de sensado y plataforma (seguro)
    start_dt = datetime.fromisoformat(start_date).date()
    end_dt = datetime.fromisoformat(end_date).date()

    final_products = []
    for p in products:
        name = p.get('Name', '')
        # 1) comprobar plataforma vía nombre (si PLATFORM fue pedido)
        if platform and not name.startswith(f"{platform}_"):
            # si no coincide, omitir
            continue
        # 2) extraer fecha de sensado desde name
        sensing_dt = _parse_sensing_datetime_from_name(name)
        if sensing_dt:
            if start_dt <= sensing_dt.date() <= end_dt:
                final_products.append(p)
        else:
            # fallback a ContentDate
            cd = p.get('ContentDate', {})
            cd_start = cd.get('Start')
            if cd_start:
                try:
                    cd_dt = datetime.fromisoformat(cd_start.replace('Z','')).date()
                    if start_dt <= cd_dt <= end_dt:
                        final_products.append(p)
                except Exception:
                    pass

    print(f"✓ After name/content-date post-filter: {len(final_products)} products")
    return final_products

def download_sentinel2_product(product_info, access_token, download_dir=DOWNLOAD_DIR, chunk_size=16*1024):
    product_id = product_info['Id']
    product_name = product_info['Name']
    download_dir = Path(download_dir)
    download_dir.mkdir(parents=True, exist_ok=True)

    download_url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
    zip_filename = download_dir / f"{product_name}.zip"
    headers = {}
    if access_token:
        headers['Authorization'] = f'Bearer {access_token}'

    if zip_filename.exists():
        print(f"    ✓ File already exists, skipping download: {zip_filename.name}")
        return str(zip_filename)

    print(f"📥 Downloading: {product_name}")
    try:
        with requests.get(download_url, headers=headers, stream=True, timeout=120) as r:
            if r.status_code != 200:
                print(f"    ✗ Download failed: {r.status_code}")
                return None
            total = int(r.headers.get('content-length', 0))
            downloaded = 0
            with open(zip_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    if chunk:
                        f.write(chunk)
                        downloaded += len(chunk)
                        if total:
                            if downloaded % (10*1024*1024) < chunk_size:
                                percent = downloaded / total * 100
                                print(f"    Progress: {percent:.1f}% ({downloaded/1024/1024:.1f} MB)")
            print(f"    ✓ Download completed: {downloaded/1024/1024:.1f} MB")
            return str(zip_filename)
    except Exception as e:
        print(f"    ✗ Error downloading: {e}")
        if zip_filename.exists():
            try:
                zip_filename.unlink()
            except:
                pass
        return None

def extract_and_process_safe(zip_file, output_dir=PROCESSED_DIR):
    zip_file = Path(zip_file)
    print(f"📦 Extracting and processing: {zip_file.name}")
    extract_dir = zip_file.with_suffix('')
    output_dir = Path(output_dir)
    extract_dir.mkdir(parents=True, exist_ok=True)
    output_dir.mkdir(parents=True, exist_ok=True)

    try:
        with zipfile.ZipFile(zip_file, 'r') as zf:
            zf.extractall(extract_dir)
    except Exception as e:
        print(f"    ✗ Error extracting zip: {e}")
        return None

    safe_dirs = list(extract_dir.glob("*.SAFE"))
    if not safe_dirs:
        safe_dirs = list(extract_dir.rglob("*.SAFE"))
    if not safe_dirs:
        print("    ✗ No .SAFE directory found")
        return None

    safe_dir = safe_dirs[0]
    safe_name = safe_dir.name
    print(f"    Processing .SAFE: {safe_name}")

    band_pattern = str(safe_dir / "GRANULE" / "*" / "IMG_DATA" / "R10m" / "*_B0?_10m.jp2")
    band_files = glob.glob(band_pattern)
    if not band_files:
        band_files = list(safe_dir.rglob("*_B0?_10m.jp2"))

    rgb_bands = {}
    for bf in band_files:
        if '_B02_' in bf:
            rgb_bands['B02'] = bf
        elif '_B03_' in bf:
            rgb_bands['B03'] = bf
        elif '_B04_' in bf:
            rgb_bands['B04'] = bf

    if len(rgb_bands) != 3:
        print(f"    ✗ Missing RGB bands, found: {list(rgb_bands.keys())}")
        return None

    output_filename = f"{safe_name.replace('.SAFE','')}_RGB_10m.tiff"
    output_path = output_dir / output_filename

    success = create_rgb_composite(rgb_bands, output_path, safe_name)
    if success:
        print(f"    ✓ RGB composite created: {output_filename}")
        return str(output_path)
    else:
        print("    ✗ Failed to create RGB composite")
        return None

def create_rgb_composite(rgb_bands, output_path, safe_name):
    try:
        bands_data = {}
        transform = None
        crs = None
        height = width = None

        for band_key in ['B04', 'B03', 'B02']:
            band_file = rgb_bands.get(band_key)
            with rasterio.open(band_file) as src:
                arr = src.read(1)
                bands_data[band_key] = arr
                if transform is None:
                    transform = src.transform
                    crs = src.crs
                    height, width = src.shape

        rgb_array = np.stack([bands_data['B04'], bands_data['B03'], bands_data['B02']], axis=0)
        rgb_normalized = np.clip(rgb_array * 3.5, 0, 10000).astype(np.uint16)

        output_path = Path(output_path)
        with rasterio.open(
            output_path, 'w',
            driver='GTiff',
            height=height,
            width=width,
            count=3,
            dtype='uint16',
            crs=crs,
            transform=transform,
            compress='lzw',
            tiled=True,
            blockxsize=512,
            blockysize=512
        ) as dst:
            dst.write(rgb_normalized)
            dst.update_tags(
                PRODUCT_NAME=safe_name,
                BANDS='RGB (B04,B03,B02)',
                RESOLUTION='10m',
                PRODUCT_TYPE='SENTINEL2_L2A_COMPLETE_TILE_RGB'
            )

        size_mb = output_path.stat().st_size / (1024*1024)
        print(f"        ✓ RGB composite saved ({size_mb:.1f} MB)")
        return True
    except Exception as e:
        print(f"        ✗ RGB creation error: {e}")
        return False

def download_complete_sentinel2_tiles_odata(aoi, start_date, end_date, platform=None, max_products=MAX_PRODUCTS_TO_PROCESS):
    print("\n" + "="*60)
    print("DOWNLOADING COMPLETE SENTINEL-2 TILES (PLATFORM FILTERING)")
    print("="*60)
    print(f"AOI: {aoi}")
    print(f"Date range: {start_date} to {end_date}")
    print(f"Platform filter: {platform}")
    print("="*60)

    token = get_cdse_access_token()
    if not token:
        print("✗ No access token. Aborting.")
        return

    products = search_sentinel2_products_odata(aoi, start_date, end_date, access_token=token, platform=platform, top=TOP_N)
    if not products:
        print("No products found after post-filter.")
        return

    if max_products:
        products = products[:max_products]

    successful = failed = 0
    for i, p in enumerate(products, 1):
        print(f"\n[{i}/{len(products)}] Processing product: {p.get('Name')}")
        zipf = download_sentinel2_product(p, token)
        if not zipf:
            failed += 1
            continue
        processed = extract_and_process_safe(zipf)
        if processed:
            successful += 1
            print(f"✓ Successfully processed: {Path(processed).name}")
        else:
            failed += 1

    print("\n" + "="*60)
    print("DOWNLOAD FINISHED")
    print("="*60)
    print(f"✓ Successfully processed: {successful}")
    print(f"✗ Failed: {failed}")
    print(f"Processed files in: {PROCESSED_DIR.resolve()}")
    print(f"Raw zips in: {DOWNLOAD_DIR.resolve()}")

if __name__ == "__main__":
    download_complete_sentinel2_tiles_odata(AOI, START_DATE, END_DATE, platform=PLATFORM, max_products=MAX_PRODUCTS_TO_PROCESS)



DOWNLOADING COMPLETE SENTINEL-2 TILES (PLATFORM FILTERING)
AOI: [-63.95, -31.75, -63.75, -31.6]
Date range: 2024-07-01 to 2024-07-20
Platform filter: S2A
✓ Access token obtained successfully
🔍 Searching for Sentinel-2 products via OData...
⚠️ OData returned non-200. Intentando fallback usando ContentDate...
✓ OData returned 4 products (before post-filtering)
✓ After name/content-date post-filter: 4 products

[1/4] Processing product: S2A_MSIL2A_20240712T140711_N0510_R110_T20HMK_20240712T213453.SAFE
    ✓ File already exists, skipping download: S2A_MSIL2A_20240712T140711_N0510_R110_T20HMK_20240712T213453.SAFE.zip
📦 Extracting and processing: S2A_MSIL2A_20240712T140711_N0510_R110_T20HMK_20240712T213453.SAFE.zip
    Processing .SAFE: S2A_MSIL2A_20240712T140711_N0510_R110_T20HMK_20240712T213453.SAFE
        ✗ RGB creation error: Deleting processed_tiles\S2A_MSIL2A_20240712T140711_N0510_R110_T20HMK_20240712T213453_RGB_10m.tiff failed: Permission denied
    ✗ Failed to create RGB composite


KeyboardInterrupt: 